In [6]:
import cv2
import numpy as np

In [7]:
def mask_from_color(image_hsv, lower, upper, erode=0, dilate=0):
    
    mask = cv2.inRange(image_hsv, lower, upper)
    
    if erode: mask = cv2.erode(mask, None, iterations=erode)
    if dilate: mask = cv2.dilate(mask, None, iterations=dilate)
    
    return mask

def resize_same(image, new_height=False, new_width=False):
    ar = image.shape[1]/image.shape[0]
    
    if new_height: 
        new_width = int(ar * new_height)
    elif new_width: 
        new_height = int(new_width/ar)
    
    return cv2.resize(image, dsize=(new_width, new_height))

In [8]:
BLUE, GREEN, RED = (255, 0, 0), (0, 255, 0), (0, 0, 255)
RED_LOWER, RED_UPPER = (0, 100, 100), (10, 230, 230)
WHITE_LOWER, WHITE_UPPER = (200, 200, 200), (255, 255, 255)
GREEN_LOWER, GREEN_UPPER = (80/2, 100, 100), (120/2, 230, 230)

cap = cv2.VideoCapture('video.mp4')

while True:
    ret, image = cap.read()
    
    if not ret: break
        
    blur = 5
    blurred = cv2.GaussianBlur(image, (blur, blur), 0)
    img = resize_same(blurred, new_width=800)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    ground_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    players_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    
    green_mask = mask_from_color(hsv, GREEN_LOWER, GREEN_UPPER, erode=2, dilate=5)
    cnts, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c = max(cnts, key=cv2.contourArea)
    cv2.drawContours(ground_mask, [c], -1, 255, -1)
    img_ground = cv2.bitwise_and(img, img, mask = ground_mask)
    
    red_mask = mask_from_color(hsv, RED_LOWER, RED_UPPER, dilate=10)
    img_red = cv2.bitwise_and(img_ground, img_ground, mask = red_mask)
    _, red_mask = cv2.threshold(cv2.cvtColor(img_red, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)
    
    white_mask = mask_from_color(hsv, WHITE_LOWER, WHITE_UPPER, erode=1, dilate=4)
    img_white = cv2.bitwise_and(img_ground, img_ground, mask = white_mask)
    _, white_mask = cv2.threshold(cv2.cvtColor(img_white, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)
    
    cnts, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        if area > 1500:
            (x, y, w, h) = cv2.boundingRect(cnt)
            cv2.rectangle(players_mask, (x, y), (x+w, y+h), 255, -1)

    cnts, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        if area > 1500:
            (x, y, w, h) = cv2.boundingRect(cnt)
            cv2.rectangle(players_mask, (x, y), (x+w, y+h), 255, -1)
            
#     output = np.vstack([np.hstack([img, img_ground]), 
#                                     np.hstack([img_red, img_white])])

    img_players = cv2.bitwise_and(img, img, mask = players_mask)
    output = np.hstack([img, img_players])
    cv2.imshow('output', output)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()